In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import eikon as ek
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import os
from PIL import Image
import mplfinance as mpf
import yfinance as yf

In [3]:
!nvidia-smi

Sun Dec  1 06:08:05 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.85                 Driver Version: 555.85         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3080      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   44C    P8             39W /  320W |     830MiB /  10240MiB |      9%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [16]:
ek.set_app_key('7001d23cf74b4dd3a4ba3ed95bba9d29fb71e7ff')

In [4]:
# Create the additional plot for the line
custom_style = mpf.make_mpf_style( marketcolors=mpf.make_marketcolors(
    up='green',  # Color for bullish candles
    down='red',  # Color for bearish candles
    wick={'up': 'green', 'down': 'red'},  # Wick colors
    edge={'up': 'green', 'down': 'red'}  # Border colors
),
                                 gridstyle = '',
    
                                 facecolor="white")


#addplot1 = mpf.make_addplot(dataqqq['Close'], type='line', color='black')
#addplot2 = mpf.make_addplot(datavix['Close'], type='line', color='black')

# Plot the candlestick chart with the line plot overlay
#mpf.plot(
#    data,
#    type='candle',
#    addplot=[addplot1, addplot2],
#    style=custom_style
#)

In [8]:
import matplotlib
matplotlib.use("Agg")  # Non-interactive backend

In [17]:
# Get ticker names in Nasdaq Composite
df, err = ek.get_data('0#.IXIC','CF_NAME')
ticker_list1 = df.iloc[:, 0].tolist()

C:\Users\rebei\anaconda3\envs\forecast\lib\site-packages\eikon\data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [24]:
# Get ticker names in NYSE Composite
df, err = ek.get_data('0#.NYA','CF_NAME')
ticker_list2 = df.iloc[:, 0].tolist()

In [164]:
def download_resize_and_crop_images(input_folder, output_folder, target_size, crop_box):
    """
    Resize all images in the input folder to a target size, then crop them and save to the output folder.

    Parameters:
    - input_folder (str): Path to the folder containing the images.
    - output_folder (str): Path to the folder where resized and cropped images will be saved.
    - target_size (tuple): Target size (width, height) to resize images.
    - crop_box (tuple): A tuple (left, upper, right, lower) defining the cropping box.
    """
    # Download historical data for a stock, e.g., Apple (AAPL)
    for tick_list in [ticker_list2_copy]:
        #for year in ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014']:
        #for year in ['2020', '2018', '2017', '2016', '2015', '2014']:
        for year in ['2024']:
            print(year)
            for i in range(len(tick_list)):
                try:
                    ticker= tick_list[i]
                    print(ticker, i)
                    # ensure the chart has 42 days/candles
                    if year == '2015' or year == '2014' or year == '2020':
                        data = yf.download(ticker, start=str(year) + '-8-12', end=str(year) + '-10-10', progress=False)
                    else: 
                        data = yf.download(ticker, start=str(year) + '-8-10', end=str(year) + '-10-10', progress=False)
                    data.drop('Adj Close', axis = 1, inplace = True)
                    data['Volume'] = data['Volume'].astype(int)
                    data['Open'] = data['Open'].astype(float)
                    data['Low'] = data['Low'].astype(float)
                    data['High'] = data['High'].astype(float)
                    data['Close'] = data['Close'].astype(float)
                    data.columns = data.columns.droplevel(level=1)
                    data.columns = data.columns.str.replace(ticker, '')  # This removes ticekr from the column names
        
                    # save plot with custom colors
                    mpf.plot(
                        data,
                        type='candle',
                        #addplot=addplot,
                        volume=False,
                        style=custom_style,
                        figsize=(8, 5),
                        savefig='charts2/'+str(ticker)+'-' + str(year) +'.png'
                    )
                    
                    gc.collect()  # Force garbage collection
        
        
                except Exception as e: 
                    print(e)
                    continue


    # Loop over all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png')):
            try:
                # Open the image
                img_path = os.path.join(input_folder, filename)
                img = Image.open(img_path)

                # Resize the image to the target size
                img_resized = img.resize(target_size)

                # Crop the image
                cropped_img = img_resized.crop(crop_box)

                # Save the cropped image
                cropped_img_path = os.path.join(output_folder, filename)
                cropped_img.save(cropped_img_path)
                print(f"Resized and cropped image saved: {cropped_img_path}")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

In [165]:
input_folder = 'charts2/'  # Replace with your folder path
output_folder = 'charts2/'  # Replace with your output folder path
target_size = (800, 500)  # Resize all images to 800x800
crop_box = (170, 60, 700, 400)  # Define your cropping box (left, upper, right, lower)
download_resize_and_crop_images(input_folder, output_folder, target_size, crop_box)

2020
UNF 0
UVV 1
DVN 2
SHEL 3
VVI 4
TRI 5
AFG 6
BBUC 7


2024-11-27 08:37:57,514 P[26224] [MainThread 5604] 
1 Failed download:
2024-11-27 08:37:57,514 P[26224] [MainThread 5604] ['BBUC']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2020-8-12 -> 2020-10-10) (Yahoo error = "Data doesn\'t exist for startDate = 1597204800, endDate = 1602302400")')


index -1 is out of bounds for axis 0 with size 0
NMR 8
VRT 9
FCPT 10
RGA 11
STAG 12
KB 13
GRNT 14


2024-11-27 08:38:00,465 P[26224] [MainThread 5604] 
1 Failed download:
2024-11-27 08:38:00,466 P[26224] [MainThread 5604] ['GRNT']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2020-8-12 -> 2020-10-10) (Yahoo error = "Data doesn\'t exist for startDate = 1597204800, endDate = 1602302400")')


index -1 is out of bounds for axis 0 with size 0
QD 15
GE 16
THC 17
BSAC 18
TYL 19
TRN 20
VTLE 21
DEI 22
CDE 23
ATHM 24
CLPR 25
MLNK 26


...

index -1 is out of bounds for axis 0 with size 0
MSB 1869
EXK 1870
AXP 1871
FBIN 1872
BHVN 1873


2024-11-27 12:39:56,544 P[26224] [MainThread 5604] 
1 Failed download:
2024-11-27 12:39:56,546 P[26224] [MainThread 5604] ['BHVN']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2014-8-12 -> 2014-10-10) (Yahoo error = "Data doesn\'t exist for startDate = 1407816000, endDate = 1412913600")')


index -1 is out of bounds for axis 0 with size 0
ACRE 1874
KAI 1875
EVR 1876
MDT 1877
CNC 1878
CAVA 1879


2024-11-27 12:40:12,439 P[26224] [MainThread 5604] 
1 Failed download:
2024-11-27 12:40:12,440 P[26224] [MainThread 5604] ['CAVA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2014-8-12 -> 2014-10-10) (Yahoo error = "Data doesn\'t exist for startDate = 1407816000, endDate = 1412913600")')


index -1 is out of bounds for axis 0 with size 0
NOV 1880
AEE 1881
DX 1882
TM 1883
Resized and cropped image saved: charts2/A-2014.png
Resized and cropped image saved: charts2/A-2015.png
Resized and cropped image saved: charts2/A-2016.png
Resized and cropped image saved: charts2/A-2017.png
Resized and cropped image saved: charts2/A-2018.png
Resized and cropped image saved: charts2/A-2020.png
Resized and cropped image saved: charts2/AA-2014.png
Resized and cropped image saved: charts2/AA-2015.png
Resized and cropped image saved: charts2/AA-2016.png
Resized and cropped image saved: charts2/AA-2017.png
Resized and cropped image saved: charts2/AA-2018.png
Resized and cropped image saved: charts2/AA-2020.png
Resized and cropped image saved: charts2/AAP-2014.png
Resized and cropped image saved: charts2/AAP-2015.png
Resized and cropped image saved: charts2/AAP-2016.png
Resized and cropped image saved: charts2/AAP-2017.png
Resized and cropped image saved: charts2/AAP-2018.png
Resized and cropp

In [6]:
import pandas as pd
dfff = pd.read_csv('nasdaq.csv', header=None)
ticker_list1_copy = dfff.iloc[:, 0].tolist()
ticker_list1_copy = ticker_list1_copy[1:]
ticker_list1_copy

['AAL',
 'AAOI',
 'AAON',
 'AAPL',
 'ABCL',
 'ABL',
 'ABLLL',
 'ABNB',
 'ABSI',
 'ABUS',
 'ABVX',
 'ACAD',
 'ACCD',
 'ACDC',
 'ACGL',
 'ACGLN',
 'ACGLO',
 'ACHC',
 'ACIC',
 'ACIU',
 'ACIW',
 'ACLS',
 'ACLX',
 'ACMR',
 'ACNB',
 'ACT',
 'ACTG',
 'ACVA',
 'ADBE',
 'ADEA',
 'ADI',
 'ADMA',
 'ADP',
 'ADPT',
 'ADSE',
 'ADSK',
 'ADTN',
 'ADUS',
 'ADV',
 'AEHR',
 'AEIS',
 'AEP',
 'AEYE',
 'AFRM',
 'AFYA',
 'AGIO',
 'AGNC',
 'AGNCL',
 'AGNCM',
 'AGNCN',
 'AGNCO',
 'AGNCP',
 'AGYS',
 'AHCO',
 'AIOT',
 'AIP',
 'AIRJ',
 'AIRS',
 'AKAM',
 'AKBA',
 'AKRO',
 'ALAB',
 'ALGM',
 'ALGN',
 'ALGT',
 'ALHC',
 'ALKS',
 'ALKT',
 'ALLO',
 'ALMS',
 'ALNT',
 'ALNY',
 'ALRM',
 'ALRS',
 'ALT',
 'ALTI',
 'ALTR',
 'ALVO',
 'ALVOW',
 'AMAL',
 'AMAT',
 'AMBA',
 'AMCX',
 'AMD',
 'AMED',
 'AMGN',
 'AMKR',
 'AMLX',
 'AMPH',
 'AMPL',
 'AMRK',
 'AMRX',
 'AMSC',
 'AMSF',
 'AMWD',
 'AMZN',
 'ANAB',
 'ANDE',
 'ANGI',
 'ANIP',
 'ANNX',
 'ANSS',
 'AOSL',
 'APA',
 'APEI',
 'APGE',
 'API',
 'APLD',
 'APLS',
 'APLT',
 'APOG',
 'AP

In [42]:
df, err = ek.get_data(instruments=".SPX", fields=["TR.IndexConstituentRIC","TR.IndexConstituentName"])

C:\Users\rebei\anaconda3\envs\forecast\lib\site-packages\eikon\data_grid.py:255: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df = df.apply(pd.to_numeric, errors='ignore')


In [43]:
spy_tickers = df['Constituent RIC'].tolist()
spy_tickers

['POOL.OQ',
 'CHRW.OQ',
 'AJG.N',
 'CNP.N',
 'AMCR.N',
 'WM.N',
 'BA.N',
 'FOX.OQ',
 'WY.N',
 'MCD.N',
 'HD.N',
 'SPG.N',
 'HBAN.OQ',
 'QRVO.OQ',
 'LVS.N',
 'CTRA.N',
 'ON.OQ',
 'AIG.N',
 'AIZ.N',
 'CARR.N',
 'EVRG.OQ',
 'SMCI.OQ',
 'COST.OQ',
 'MCO.N',
 'DIS.N',
 'LULU.OQ',
 'PAYX.OQ',
 'AMD.OQ',
 'REG.OQ',
 'DHI.N',
 'TRV.N',
 'MOS.N',
 'BWA.N',
 'WDC.OQ',
 'VTR.N',
 'IVZ.N',
 'ZBRA.OQ',
 'AMAT.OQ',
 'STX.OQ',
 'FDS.N',
 'KVUE.N',
 'SYK.N',
 'DELL.N',
 'VRSN.OQ',
 'TECH.OQ',
 'NVR.N',
 'JBL.N',
 'LOW.N',
 'BSX.N',
 'SYY.N',
 'FCX.N',
 'MAS.N',
 'BEN.N',
 'META.OQ',
 'PAYC.N',
 'EXPE.OQ',
 'AMTM.N',
 'RJF.N',
 'CL.N',
 'VICI.N',
 'CE.N',
 'UNH.N',
 'ARE.N',
 'GEN.OQ',
 'LLY.N',
 'DAY.N',
 'GRMN.N',
 'PODD.OQ',
 'ISRG.OQ',
 'AVB.N',
 'TPR.N',
 'CBRE.N',
 'CINF.OQ',
 'WMB.N',
 'EA.OQ',
 'TMUS.OQ',
 'CB.N',
 'TXN.OQ',
 'MDLZ.OQ',
 'BRO.N',
 'BALL.N',
 'PFG.OQ',
 'KEYS.N',
 'PCG.N',
 'RL.N',
 'JPM.N',
 'TTWO.OQ',
 'CDW.OQ',
 'RMD.N',
 'LNT.OQ',
 'COO.OQ',
 'CMG.N',
 'PCAR.OQ',
 'GNRC.N',


In [57]:
def download_resize_and_crop_images(input_folder, output_folder, target_size, crop_box):
    """
    Resize all images in the input folder to a target size, then crop them and save to the output folder.

    Parameters:
    - input_folder (str): Path to the folder containing the images.
    - output_folder (str): Path to the folder where resized and cropped images will be saved.
    - target_size (tuple): Target size (width, height) to resize images.
    - crop_box (tuple): A tuple (left, upper, right, lower) defining the cropping box.
    """
    # Download historical data for a stock, e.g., Apple (AAPL)
    for tick_list in [spy_tickers]:
        #for year in ['2023', '2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014']:
        #for year in ['2018', '2017', '2016', '2015', '2014']:
        for year in ['2013']:
            print(year)
            for i in range(len(tick_list)):
                try:
                    ticker= tick_list[i]
                    print(ticker, i)
                    data = ek.get_timeseries(ticker,
                                            fields = '*', 
                                            start_date = str(year) + '-8-13',
                                            end_date= str(year) + '-10-10')
        
                    data = data.drop(columns=["COUNT"])
                    data = data.rename(columns={"HIGH": "High", "LOW": "Low", "OPEN": "Open", "VOLUME": "Volume", "CLOSE": "Close" })
                    data['Volume'] = data['Volume'].astype(int)
                    data['Open'] = data['Open'].astype(float)
                    data['Low'] = data['Low'].astype(float)
                    data['High'] = data['High'].astype(float)
                    data['Close'] = data['Close'].astype(float)
        
                    #datavix = ek.get_timeseries('.VIX',
                    #        fields = '*', 
                    #        start_date = '2023-6-10',
                    #        end_date= '2023-8-10')
        
        
                    
                    #datavix = datavix.rename(columns={"HIGH": "High", "LOW": "Low", "OPEN": "Open", "VOLUME": "Volume", "CLOSE": "Close" })
        
                    # save plot with custom colors
                    mpf.plot(
                        data,
                        type='candle',
                        #addplot=addplot,
                        volume=False,
                        style=custom_style,
                        figsize=(8, 5),
                        savefig='chartsfinal/'+str(ticker)+'-' + str(year) +'.png'
                    )
                    
                    gc.collect()  # Force garbage collection
        
        
                except Exception as e: 
                    print(e)
                    continue




    # Loop over all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(('.png')):
            try:
                # Open the image
                img_path = os.path.join(input_folder, filename)
                img = Image.open(img_path)

                # Resize the image to the target size
                img_resized = img.resize(target_size)

                # Crop the image
                cropped_img = img_resized.crop(crop_box)

                # Save the cropped image
                cropped_img_path = os.path.join(output_folder, filename)
                cropped_img.save(cropped_img_path)
                print(f"Resized and cropped image saved: {cropped_img_path}")

            except Exception as e:
                print(f"Error processing {filename}: {e}")

In [58]:
input_folder = 'chartsfinal/'  # Replace with your folder path
output_folder = 'chartsfinal/'  # Replace with your output folder path
target_size = (800, 500)  # Resize all images to 800x800
crop_box = (170, 60, 700, 400)  # Define your cropping box (left, upper, right, lower)
download_resize_and_crop_images(input_folder, output_folder, target_size, crop_box)

2013
POOL.OQ 0
CHRW.OQ 1
AJG.N 2
CNP.N 3


2024-12-02 13:45:17,976 P[12040] [MainThread 1692] Error with AMCR.N: No data available for the requested date range
2024-12-02 13:45:17,977 P[12040] [MainThread 1692] AMCR.N: No data available for the requested date range | 


AMCR.N 4
Error code -1 | AMCR.N: No data available for the requested date range | 
WM.N 5
BA.N 6


2024-12-02 13:45:19,269 P[12040] [MainThread 1692] Error with FOX.OQ: No data available for the requested date range
2024-12-02 13:45:19,269 P[12040] [MainThread 1692] FOX.OQ: No data available for the requested date range | 


FOX.OQ 7
Error code -1 | FOX.OQ: No data available for the requested date range | 
WY.N 8
MCD.N 9
HD.N 10
SPG.N 11
HBAN.OQ 12
QRVO.OQ 13
LVS.N 14
CTRA.N 15
ON.OQ 16
AIG.N 17
AIZ.N 18
CARR.N 19


...

DOW.N 479
Error code -1 | DOW.N: No data available for the requested date range | 
PPL.N 480
NTRS.OQ 481
ETN.N 482
CVX.N 483
CI.N 484
ACN.N 485


2024-12-02 13:50:12,321 P[12040] [MainThread 1692] Error with CRWD.OQ: No data available for the requested date range
2024-12-02 13:50:12,321 P[12040] [MainThread 1692] CRWD.OQ: No data available for the requested date range | 


CRWD.OQ 486
Error code -1 | CRWD.OQ: No data available for the requested date range | 
PANW.OQ 487
XYL.N 488
HAS.OQ 489
ENPH.OQ 490
SO.N 491


2024-12-02 13:50:16,118 P[12040] [MainThread 1692] Error with CTLT.N: No data available for the requested date range
2024-12-02 13:50:16,118 P[12040] [MainThread 1692] CTLT.N: No data available for the requested date range | 


CTLT.N 492
Error code -1 | CTLT.N: No data available for the requested date range | 
EQT.N 493
NRG.N 494
MKTX.OQ 495
LMT.N 496
DLR.N 497
PSX.N 498
SCHW.N 499
SNPS.OQ 500
J.N 501
AVY.N 502
Resized and cropped image saved: chartsfinal/A.N-2013.png
Resized and cropped image saved: chartsfinal/AAPL.OQ-2013.png
Resized and cropped image saved: chartsfinal/ABBV.N-2013.png
Resized and cropped image saved: chartsfinal/ABT.N-2013.png
Resized and cropped image saved: chartsfinal/ACGL.OQ-2013.png
Resized and cropped image saved: chartsfinal/ACN.N-2013.png
Resized and cropped image saved: chartsfinal/ADBE.OQ-2013.png
Resized and cropped image saved: chartsfinal/ADI.OQ-2013.png
Resized and cropped image saved: chartsfinal/ADM.N-2013.png
Resized and cropped image saved: chartsfinal/ADP.OQ-2013.png
Resized and cropped image saved: chartsfinal/ADSK.OQ-2013.png
Resized and cropped image saved: chartsfinal/AEE.N-2013.png
Resized and cropped image saved: chartsfinal/AEP.OQ-2013.png
Resized and cropped im